# Spectrum 101

Here you will learn how to obtain field-free and -dressed molecular spectra. The spectrum utility needs to be imported:

In [1]:
from richmol.utils import spectrum #TODO

## Field-free spectrum

The hyperfine field-free spectrum of the electric dipole moment of water is obtained.

### Initialization

To start, energies of field-free states and matrix elements of field tensor operators of interest are needed (see `Tensors 101`). In this example necessary tensor operators are read from previously computed old-format Richmol files. For a correct `field_free_spectrum` object initialization the keyword arguments `type` (coupling moment type), `order` (coupling moment order) and `units` (coupling moment units) can be used. These keyword arguments are necessary to apply the right selection rules, absorption intensity function and unit conversion factors during computations. To narrow down molecular states of interest, the keyword arguments `j_max` (maxiumum value of J / F quantum number) and `e_max` (maximum value of energy) are available:

In [2]:
# INITIALIZATION

path = 'etc/richmol_files_water'
filename = path + '/hyfor_energies_f0.0_f39.0.chk'
matelem = path + '/matelem_MU_ns_f<f1>_f<f2>.rchm'

free_spec = spectrum.field_free_spectrum(filename, matelem=matelem, type='elec', order='dip', units='Debye', j_max=2, e_max=15e3)

When intializing a `field_free_spectrum` object from a new-format Richmol file, the keyword argument `matelem` can be ignored. Intialization automatically creates a path to an output file, into which all upcoming results are written.

### Linestrength

To obtain the linestrengths call the `linestr()` function on the `field_free_spectrum` object. Use the keywword argument `thresh` to define the linestrength threshold below which to neglect transitions:

In [3]:
# LINESTRENGTH
free_spec.linestr(thresh=1e-20)


  COMPUTED ASSIGNMENTS ...

      filename = etc/richmol_files_water/hyfor_energies_f0.0_f39.0.chk
      j_min = 0.0
      j_max = 2.0
      e_max = 15000.0

  COMPUTED LINESTRENGTHS ...

      matelem = etc/richmol_files_water/matelem_MU_ns_f<f1>_f<f2>.rchm
      type = elec
      order = dip
      linestr_thresh = 1e-20


Results are directly written into the output file.

### Absorption intensity

Finally the absorption intensities can be computed by calling the function `abs_intens()` on the `field_free_spectrum` object. Use the keyword arguments `abun` (natural terrestrial isotopic abundance), `temp` (temperature), `part_sum` (total internal partition sum) and `thresh` (absorption intensity threshold below which to neglect transitions) to obtain the solution of your problem of interest. In this step transitions and their corresponding absorption intensities can further be filtered to ones liking (e.g. into transitions, which are allowed / forbidden w.r.t. nuclear spin). To do so, just pass a list of filtering functions, which make use of the keyword arguments `sym` (total symmetry of states) and `qstr` (string containing other physical quanities of states), as keyword argument `filters`. By default no filter will be applied:

In [4]:
# ABSORPTION INTENSITY

temp = 296.0
part_sum = 174.5813

def allowed(**kwargs):
    """ Filters out transition, which are allowed w.r.t. nuclear spin. """
    pass_ns = True
    if 'qstr' in kwargs:
        qstr_bra, qstr_ket = kwargs['qstr']
        ns_bra = float(qstr_bra.split()[-1])
        ns_ket = float(qstr_ket.split()[-1])
        pass_ns = (ns_bra, ns_ket) in [(0.0, 0.0), (1.0, 1.0)]
    return pass_ns

def forbidden(**kwargs):
    """ Filters out transition, which are forbidden w.r.t. nuclear spin. """
    pass_ns = True
    if 'qstr' in kwargs:
        qstr_bra, qstr_ket = kwargs['qstr']
        ns_bra = float(qstr_bra.split()[-1])
        ns_ket = float(qstr_ket.split()[-1])
        pass_ns = (ns_bra, ns_ket) in [(0.0, 1.0), (1.0, 0.0)]
    return pass_ns

free_spec.abs_intens(temp, part_sum, abun=1.0, filters=[allowed, forbidden] , thresh=1e-36)


  COMPUTED ABSORPTION INTENSITIES ...

      type = elec
      order = dip
      units = Debye
      abun = 1.0
      temp = 296.0
      part_sum = 174.5813

  FILTERED ABSORPTION INTENSITIES ...

      abs_intens_thresh = 1e-36
      filters = ['allowed', 'forbidden']

  PLOTTED ABSORPTION INTENSITIES ...

      abs_intens_thresh = 1e-36
      filters = ['allowed', 'forbidden']



Results are directly written into the output file. A spectrum plot `absorption_intensities.png`, containing filtered absorption intensities, will be saved into the current working directory:

![Field-free spectrum](absorption_intensities.png)